In [40]:
#use classicalvenv
import dimod
import json
import numpy as np
from config import problem_configs
import os
import glob

In [41]:
def load_ising(file_path):
    with open(file_path, "r") as f:
        all_isings_data = json.load(f)  
    return all_isings_data

def save_solutions_to_json(solutions_dict, output_filepath):
    output_list = []
    for instance_id, solutions_list in solutions_dict.items():            
        solutions = [sol[0] for sol in solutions_list]
        
        # The cost is the same for all solutions of an instance
        cost = solutions_list[0][1]

        formatted_entry = {
            "instance_id": instance_id,
            "solutions": solutions, # Stored as a list of strings
            "cost": float(cost)
        }
        
        output_list.append(formatted_entry)
        
    with open(output_filepath, 'w') as f:
        json.dump(output_list, f, indent=2)

def convert_to_dimod_ising(all_isings_data):
    dimod_ising = {}
    for instance in all_isings_data:
        instance_id = instance["instance_id"]
        terms = instance["terms"]
        weights = instance["weights"]

        linear_bias = {}
        quadratic_bias = {}

        for term, weight in zip(terms, weights):
            if len(term) == 1:
                variable = f'x{term[0]}'
                linear_bias[variable] = weight
            elif len(term) == 2:
                var1 = f'x{term[0]}'
                var2 = f'x{term[1]}'
                quadratic_bias[(var1, var2)] = weight

        dimod_ising[instance_id] = (linear_bias, quadratic_bias)
    return dimod_ising

def extract_lowest_energy_solutions(allSolutions):
    results = []
    lowest_energy = allSolutions.first.energy
    for datum in allSolutions.data():
        if datum.energy > lowest_energy:
            break
        sorted_keys = sorted(datum.sample.keys(), key=lambda v: int(v[1:]))
        variable_values = []
        for key in sorted_keys:
            val = datum.sample[key]
            if val > 0:
                variable_values.append(f"+{val}")
            else:
                variable_values.append(str(val))
        
        variable_string = ",".join(variable_values)
        energy = float(datum.energy)
        results.append((variable_string, energy))
            
    return results

In [42]:
#////////// variables //////////
#isingBatchFile = "batch_Ising_data_Knapsack_6_items_9q_.json"
#isingBatchFile = "batch_Ising_data_TSP_9q_.json"
# isingBatchFile = "batch_Ising_data_MinimumVertexCover_9q_.json"

FILEDIRECTORY = "isingBatches/"

In [43]:
for problem_name, config in problem_configs.items():
    print(f"--- Processing class: {problem_name} ---")

    file_slug = config['file_slug']
    search_pattern = os.path.join(FILEDIRECTORY, f"batch*{file_slug}.json")
    isingBatchFile = glob.glob(search_pattern)[0]

    isings = load_ising(isingBatchFile)
    dimodIsings = convert_to_dimod_ising(isings)
    #print(dimodIsings)

    solver = dimod.ExactSolver()

    globalOptima = {}
    for instance, biases in dimodIsings.items():
        #print(instance)
        I = dimodIsings[instance]
        sampleset = solver.sample_ising(I[0], I[1])
        instanceGlobalOptima = extract_lowest_energy_solutions(sampleset)
        globalOptima[instance] = instanceGlobalOptima

    #print(globalOptima)
    print('Top 10 solutions for the final problem instance \/ \/ \/')
    print(sampleset.slice(10))

    # Split the path into directory and filename
    directory, filename = os.path.split(isingBatchFile)

    # Construct the new path by adding 'solved_' to the filename
    output_file = os.path.join(directory, f"solved_{filename}")
    save_solutions_to_json(globalOptima, output_file)

--- Processing class: TSP ---
Top 10 solutions for the final problem instance \/ \/ \/
  x0 x1 x2 x3 x4 x5 x6 x7 x8 energy num_oc.
0 +1 -1 +1 -1 +1 +1 +1 +1 -1 -107.0       1
1 +1 +1 -1 -1 +1 +1 +1 -1 +1 -107.0       1
2 +1 -1 +1 +1 +1 -1 -1 +1 +1 -105.0       1
3 -1 +1 +1 +1 +1 -1 +1 -1 +1 -105.0       1
4 -1 +1 +1 +1 -1 +1 +1 +1 -1  -95.0       1
5 +1 +1 -1 +1 -1 +1 -1 +1 +1  -95.0       1
6 +1 -1 +1 +1 +1 +1 -1 +1 +1  -89.0       1
7 -1 +1 +1 +1 +1 +1 +1 -1 +1  -89.0       1
8 +1 -1 +1 -1 +1 +1 +1 +1 +1  -89.0       1
9 +1 +1 +1 -1 +1 +1 +1 -1 +1  -89.0       1
['SPIN', 10 rows, 10 samples, 9 variables]
--- Processing class: Knapsack ---
Top 10 solutions for the final problem instance \/ \/ \/
  x0 x1 x2 x3 x4 x5 x6 x7 x8 energy num_oc.
0 +1 +1 +1 +1 -1 -1 -1 +1 +1 -380.0       1
1 +1 +1 +1 +1 -1 +1 -1 -1 +1 -380.0       1
2 -1 +1 +1 -1 -1 +1 +1 +1 +1 -379.0       1
3 +1 +1 +1 +1 +1 -1 +1 -1 +1 -378.0       1
4 +1 +1 +1 -1 +1 +1 -1 +1 +1 -378.0       1
5 +1 -1 +1 +1 -1 +1 +1 -1 +1 -

In [44]:
# print(sampleset.slice(10))
# print(sampleset)